In [1]:
import pandas as pd 
import numpy as np
import os
from fmpapi import fmp_get
from dotenv import load_dotenv
import pyarrow

load_dotenv()
api_key = os.getenv('FMP_API_KEY')


In [2]:
bs = fmp_get(

  resource="balance-sheet-statement", 
  symbol="AAPL", 
  params={"period": "annual", "limit": 5},
  to_pandas=True
)

In [ ]:
cf = fmp_get(
    resource='cash-flow-statement',
    symbol='AAPL',
    params={'period':'annual', 'limit':5},
    to_pandas=True
)

In [5]:
cf

,date,symbol,reported_currency,cik,filling_date,accepted_date,calendar_year,period,net_income,depreciation_and_amortization,...,net_cash_used_provided_by_financing_activities,effect_of_forex_changes_on_cash,net_change_in_cash,cash_at_end_of_period,cash_at_beginning_of_period,operating_cash_flow,capital_expenditure,free_cash_flow,link,final_link
0,2024-09-28,AAPL,USD,0000320193,2024-11-01,2024-11-01 06:01:36,2024,FY,93736000000,11445000000,...,-121983000000,0,-794000000,29943000000,30737000000,118254000000,-9447000000,108807000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
1,2023-09-30,AAPL,USD,0000320193,2023-11-03,2023-11-02 18:08:27,2023,FY,96995000000,11519000000,...,-108488000000,0,5760000000,30737000000,24977000000,110543000000,-10959000000,99584000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
2,2022-09-24,AAPL,USD,0000320193,2022-10-28,2022-10-27 18:01:14,2022,FY,99803000000,11104000000,...,-110749000000,0,-10952000000,24977000000,35929000000,122151000000,-10708000000,111443000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
3,2021-09-25,AAPL,USD,0000320193,2021-10-29,2021-10-28 18:04:28,2021,FY,94680000000,11284000000,...,-93353000000,0,-3860000000,35929000000,39789000000,104038000000,-11085000000,92953000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
4,2020-09-26,AAPL,USD,0000320193,2020-10-30,2020-10-29 18:06:25,2020,FY,57411000000,11056000000,...,-86820000000,0,-10435000000,39789000000,50224000000,80674000000,-7309000000,73365000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...


In [21]:
cf_cfo = cf['operating_cash_flow']
cf_capex = cf['capital_expenditure']
cf_debt_taken = cf['sales_maturities_of_investments']
cf_debt_repaid = cf['debt_repayment']
fcf = cf_cfo + cf_capex + cf_debt_taken + cf_debt_repaid
fcf_df = pd.DataFrame(fcf)
fcf_df.columns = ['free_cash_flow']
fcf_df

,free_cash_flow
0,165155000000
1,135197000000
2,178683000000
3,212101000000
4,196255000000


In [41]:
today_fcf = np.average(fcf_df['free_cash_flow'])
today_fcf

np.float64(177478200000.0)

In [24]:
roe = cf['net_income'] / bs['total_equity']
roe = pd.DataFrame(roe)
roe.columns = ['roe']
roe

,roe
0,1.645935
1,1.560760
2,1.969589
3,1.500713
4,0.878664


In [31]:
avg_roe = np.average(roe['roe'])
avg_roe


np.float64(1.5111321511516058)

In [35]:
reinv_rate = (cf['net_income'] + cf ['dividends_paid']) / cf['net_income']
reinv_rate = pd.DataFrame(reinv_rate)
reinv_rate.columns = ['reinv_rate']
reinv_rate_avg = np.average(reinv_rate['reinv_rate'])
reinv_rate_avg

np.float64(0.8271612810789165)

In [39]:
growth = avg_roe * reinv_rate_avg
real_growth = 0
if growth > 0.05:
    real_growth = 0.05
else:
    pass


In [38]:
cost_e = 0.1

In [40]:
real_growth

0.05

In [44]:
dcf = (today_fcf * (1+real_growth)) + ((today_fcf * ((1+real_growth)**2))/(cost_e - real_growth))
dcf

np.float64(4099746420000.0)

In [45]:
market_cap = fmp_get(
    resource='market-capitalization',
    symbol='AAPL',
    params= {'date': "2025-09-19"},
    to_pandas=True
)

In [46]:
market_cap

,symbol,date,market_cap
0,AAPL,2025-09-19,3643315745000


In [48]:
mos = dcf / market_cap['market_cap']
mos

0    1.125279
Name: market_cap, dtype: float64